In [ ]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from textblob import TextBlob
import tweepy
import os
from langchain.chat_models import ChatOpenAI

In [ ]:
# Twitter API Setup (Replace with your credentials) ---
TWITTER_API_KEY = "YOUR_API_KEY"
TWITTER_API_SECRET = "YOUR_API_SECRET"
TWITTER_ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
TWITTER_ACCESS_SECRET = "YOUR_ACCESS_SECRET"

In [ ]:
auth = tweepy.OAuth1UserHandler(
    TWITTER_API_KEY, TWITTER_API_SECRET, TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_SECRET
)
twitter_api = tweepy.API(auth)

In [ ]:
auth = tweepy.OAuth1UserHandler(
    TWITTER_API_KEY, TWITTER_API_SECRET, TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_SECRET
)
twitter_api = tweepy.API(auth)

In [ ]:
# Your API key from the OpenAI website.
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
llm = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["tweet", "name", "designation", "followers", "following", "nationality", "sentiment"],
    template="""
Social media user {name} ({designation}) tweeted that {tweet}. The user has {followers} followers and follows {following} accounts. Nationality: {nationality}. Sentiment: {sentiment}.
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
def get_user_info(username):
    try:
        user = twitter_api.get_user(screen_name=username)
        designation = user.description or "N/F"
        location = user.location or "N/F"
        nationality = location if location else "N/F"
        return {
            "followers": user.followers_count,
            "following": user.friends_count,
            "designation": designation,
            "nationality": nationality
        }
    except Exception as e:
        return {
            "followers": "N/F",
            "following": "N/F",
            "designation": "N/F",
            "nationality": "N/F"
        }

In [ ]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

In [ ]:
def analyze_tweet(username, tweet_text):
    user_info = get_user_info(username)
    sentiment = analyze_sentiment(tweet_text)

    formatted_output = chain.run(
        tweet=tweet_text,
        name=username,
        designation=user_info["designation"],
        followers=user_info["followers"],
        following=user_info["following"],
        nationality=user_info["nationality"],
        sentiment=sentiment
    )
    return formatted_output

In [ ]:
if __name__ == "__main__":
    tweet1 = "Women in Silchar Gathers protesting against arbitrary arrest amidst govt.'s \"Child Marriage Crackdown\"."
    tweet2 = "India's Supreme Court agrees to remove remarks describing Sikkimese-Nepalese as people of \"foreign origin\" from the recently-passed judgment in \"Association of Old Settlers of Sikkim vs Union of India\". This comes after days of protests & subsequent legal intervention."

    print(analyze_tweet("AnilJha", tweet1))
    print(analyze_tweet("NehaJournalist", tweet2))